---
# 単一ファイルを過去に戻すマニュアル（git restore 版）

## 0) 事前チェック（安全のため）

```bash
git status
git branch       # いまのブランチを確認（* が付いているのが現在地。今回は main を想定）
```

未コミットの変更があるなら一時退避：

```bash
git stash -u     # 迷ったら入れてOK。後で `git stash pop`
```

---

## A. 直前の `git pull` の **前** に、そのファイルだけ戻したい（最短ルート）

### ステップ1：ブランチを main に

```bash
git switch main
```

### ステップ2：戻す前に“確認”

```bash
# 直前（pull前）との違いを確認
git diff HEAD@{1} -- ファイル名
# そのファイルの履歴だけざっと見る
git log --oneline -- ファイル名
```

> `HEAD@{1}` = 直前の位置（たいてい「pull前」）。
> もし直前がpullじゃなさそうなら、後述のBに進んで特定のコミットを選ぶ。

### ステップ3：実際に戻す（作業ツリーを戻す）

```bash
git restore --source=HEAD@{1} -- ファイル名
```

> ポイント：`-- ファイル名` の **前の半角スペース**を忘れない！
> これを忘れると `unknown option 'ファイル名'` エラーになるよ。

### ステップ4：ステージしてコミット & push

```bash
git add aiu
git commit -m "Revert aiu to pre-pull (empty)"
git push origin main
```

### ステップ5：チェック

```bash
cat ファイル名                        # 中身を直接確認（空に戻っているはず）
git log --oneline -- ファイル名       # そのファイルにRevertコミットが載っているか
```

---

## B. 「pull前」ではなく、**好きな過去コミット**の状態に戻したい

### ステップ1：対象コミットを探す

```bash
git log --oneline -- ファイル名
```

出てきた行の左の短いSHA（例：`abc1234`）をメモ。

### ステップ2：その時点へ戻す（ファイルのみ）

```bash
git restore --source=abc1234 -- ファイル名
git add aiu
git commit -m "Revert aiu to state at abc1234"
git push origin main
```

### ステップ3：チェック

```bash
git show HEAD -- ファイル名           # 直近コミット時点の aiu の中身確認
```

---

## C. `HEAD@{1}` が正しいか不安なら（より確実に“pull前”を特定）

```bash
git reflog -n 10
```

出力内に `pull: Fast-forward` などが見えるはず。
その **1つ前**にあるエントリのSHAを使って：

```bash
git restore --source=<そのSHA> -- ファイル名
git add aiu
git commit -m "Revert aiu to pre-pull via reflog"
git push origin main
```

---

## よくあるつまずき＆コツ

* **`-- ファイル名` の前のスペースが必須**。スペースなしだと “unknown option”。
* `git restore` はデフォルトで **作業ツリーのみ**を変更。コミットするには **`git add` が必要**。
* Gitのバージョンでオプション表記が違うことあり（`--staged`= `-S`, `--worktree` = `-W`）。
  今回は単一ファイルなので `git restore --source=... -- ファイル名` → `git add` の流れがシンプル。

* **丸ごとpull前に戻す**なら（ファイル単位じゃなく全体）

  ```bash
  git reset --hard ORIG_HEAD    # 直前操作（pull）前の全体に戻す。未コミットは消えるので注意！
  ```

  ただしこれは“全部”戻るので、単一ファイルだけ戻したいときは使わない。

---

## まとめ（コピペ用最小セット：直前pull前に戻す）

```bash
git switch main
git diff HEAD@{1} -- ファイル名
git log --oneline -- aiu
git restore --source=HEAD@{1} -- aiu
git add aiu
git commit -m "Revert aiu to pre-pull (empty)"
git push origin main
cat aiu
git log --oneline -- aiu
```

これで、今回みたいな「work-hiroで編集された `aiu` を、main側では**空ファイル**（pull前）に戻す」が毎回同じ手順で安全にできます。
他のファイル名でも、`aiu` の部分を置き換えればOK！


In [ ]:
いいね！じゃあ「**pullで進んだあとに、特定のファイルだけ“pull前（または好きな過去）に戻す”**」ための超初心者向けマニュアルを置いておくね。コピペで使える形にしてるよ。

---

# 単一ファイルを過去に戻すマニュアル（git restore 版）

## 0) 事前チェック（安全のため）

```bash
git status
git branch       # いまのブランチを確認（* が付いているのが現在地。今回は main を想定）
```

未コミットの変更があるなら一時退避：

```bash
git stash -u     # 迷ったら入れてOK。後で `git stash pop`
```

---

## A. 直前の `git pull` の **前** に、そのファイルだけ戻したい（最短ルート）

### ステップ1：ブランチを main に

```bash
git switch main
```

### ステップ2：戻す前に“確認”

```bash
# 直前（pull前）との違いを確認
git diff HEAD@{1} -- aiu
# そのファイルの履歴だけざっと見る
git log --oneline -- aiu
```

> `HEAD@{1}` = 直前の位置（たいてい「pull前」）。
> もし直前がpullじゃなさそうなら、後述のBに進んで特定のコミットを選ぶ。

### ステップ3：実際に戻す（作業ツリーを戻す）

```bash
git restore --source=HEAD@{1} -- aiu
```

> ポイント：`-- aiu` の **前の半角スペース**を忘れない！
> これを忘れると `unknown option 'aiu'` エラーになるよ。

### ステップ4：ステージしてコミット & push

```bash
git add aiu
git commit -m "Revert aiu to pre-pull (empty)"
git push origin main
```

### ステップ5：チェック

```bash
cat aiu                        # 中身を直接確認（空に戻っているはず）
git log --oneline -- aiu       # そのファイルにRevertコミットが載っているか
```

> 旧いGitで `git restore` が使いづらい場合は代わりに：
>
> ```bash
> git checkout HEAD@{1} -- aiu
> git add aiu
> git commit -m "Revert aiu to pre-pull (empty)"
> git push origin main
> ```

---

## B. 「pull前」ではなく、**好きな過去コミット**の状態に戻したい

### ステップ1：対象コミットを探す

```bash
git log --oneline -- aiu
```

出てきた行の左の短いSHA（例：`abc1234`）をメモ。

### ステップ2：その時点へ戻す（ファイルのみ）

```bash
git restore --source=abc1234 -- aiu
git add aiu
git commit -m "Revert aiu to state at abc1234"
git push origin main
```

### ステップ3：チェック

```bash
git show HEAD -- aiu           # 直近コミット時点の aiu の中身確認
```

---

## C. `HEAD@{1}` が正しいか不安なら（より確実に“pull前”を特定）

```bash
git reflog -n 10
```

出力内に `pull: Fast-forward` などが見えるはず。
その **1つ前**にあるエントリのSHAを使って：

```bash
git restore --source=<そのSHA> -- aiu
git add aiu
git commit -m "Revert aiu to pre-pull via reflog"
git push origin main
```

---

## よくあるつまずき＆コツ

* **`-- aiu` の前のスペースが必須**。スペースなしだと “unknown option”。
* `git restore` はデフォルトで **作業ツリーのみ**を変更。コミットするには **`git add` が必要**。
* Gitのバージョンでオプション表記が違うことあり（`--staged`= `-S`, `--worktree` = `-W`）。
  今回は単一ファイルなので `git restore --source=... -- aiu` → `git add` の流れがシンプル。
* **丸ごとpull前に戻す**なら（ファイル単位じゃなく全体）

  ```bash
  git reset --hard ORIG_HEAD    # 直前操作（pull）前の全体に戻す。未コミットは消えるので注意！
  ```

  ただしこれは“全部”戻るので、単一ファイルだけ戻したいときは使わない。

---

## まとめ（コピペ用最小セット：直前pull前に戻す）

```bash
git switch main
git diff HEAD@{1} -- aiu
git log --oneline -- aiu
git restore --source=HEAD@{1} -- aiu
git add aiu
git commit -m "Revert aiu to pre-pull (empty)"
git push origin main
cat aiu
git log --oneline -- aiu
```

これで、今回みたいな「work-hiroで編集された `aiu` を、main側では**空ファイル**（pull前）に戻す」が毎回同じ手順で安全にできます。
他のファイル名でも、`aiu` の部分を置き換えればOK！
